In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# Install FastTENET via PyPI

In [1]:
!pip install -U fasttenet

In [2]:
import torch

In [3]:
print(torch.cuda.get_device_name(device = 0))

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
print(torch.cuda.device_count())

In [ ]:
import fasttenet

# Download tutorial data from GitHub



In [ ]:
!git clone https://github.com/cxinsys/fasttenet.git

# Run FastTENET with YAML config file

## Configuration parameters

### Parameters for initializing file path
- **DROOT**: root path of files
- **FPATH_EXP**: file name of expression data
- **FPATH_TRJ**: file name of pseudo time data
- **FPATH_BRANCH**: file name of cell select data
- **FPATH_TF**: file name of TF data, optional
- **SPATH_RESULT**: expected file name of result matrix data
- **MAKE_BINARY**: boolean, make binary file of expression data

### Parameters for FastTENET algorithm
- **BACKEND**: accleration framework, ex. "cpu", "gpu", "torch", "lightning", "jax", "cupy", "tf"
- **DEVICE_IDS**: list or number of devcies to use, optional, default: [0] (cpu), [list of whole gpu devices] (gpu)
- **PROCS_PER_DEVICE**: The number of processes to create per device when using non 'cpu' devices, optional, default: 1
- **BATCH_SIZE**: number of gene pairs
- **BINNINGMETHOD**: discretization method for expression values, optional, 'FSBW-L' is recommended to achieve results similar to TENET.
- **KP**: kernel coefficient, optional, default: 0.5
- **SMOOTHINGOPT**: smoothing method for expression values, optional

In [ ]:
cd fasttenet/tutorials

In [ ]:
!python tutorial_config.py --config ../configs/config_tuck_sub.yml

# Run FastTENET with utilizing FastTENET class

In [ ]:
import os
import os.path as osp

import numpy as np
from omegaconf import OmegaConf

In [ ]:
import fasttenet as fte

### utilize configuration file

In [ ]:
fpath_config = "../configs/config_tuck_sub.yml"

In [ ]:
conf = OmegaConf.load(fpath_config)

In [ ]:
conf

{'INIT': {'DROOT': './', 'FPATH_EXP': 'expression_dataTuck_sub.csv', 'FPATH_TRJ': 'pseudotimeTuck.txt', 'FPATH_BRANCH': 'cell_selectTuck.txt', 'FPATH_TF': 'mouse_tfs.txt', 'SPATH_RESULT': 'TE_result_matrix.txt', 'MAKE_BINARY': True}, 'BACKEND': 'gpu', 'DEVICE_IDS': 1, 'PROCS_PER_DEVICE': 1, 'BATCH_SIZE': '2**16', 'BINNINGMETHOD': 'FSBW-L', 'KP': 0.5, 'SMOOTHINGOPT': 'None'}

In [ ]:
worker = fte.FastTENET(config=conf)

Binary file already exist, load data from binary file


In [ ]:
result_matrix = worker.run(config=conf)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


[DISCRETIZER: FSBW-L, SMOOTHER: None]


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Save result matrix: ./TE_result_matrix.txt


In [ ]:
print(result_matrix.shape)

(3281, 3281)


### enter parameters directly

#### Parameters for initializing file path
- **dpath_exp_data**: expression data path, required
- **dpath_trj_data**: trajectory data path, required
- **dpath_branch_data**: branch(cell select) data path, required
- **dpath_tf_data**: tf data path, required
- **spath_result_matrix**: result matrix data path, optional, default: None
- **make_binary**: if True, make binary expression and node name file, optional, default: False

#### Parameters for FastTENET algorithm
- **backend**: optional, default: 'cpu'
- **device_ids**: list or number of devcies to use, optional, default: [0] (cpu), [list of whole gpu devices] (gpu)
- **procs_per_device**: The number of processes to create per device when using non 'cpu' devices, optional, default: 1
- **batch_size**: Required
- **kp**: kernel percentile, optional, default: 0.5
- **binning_method**: discretization method for expression values, optional, 'FSBW-L' is recommended to achieve results similar to TENET.


In [ ]:
droot = './'
dpath_exp_data = osp.join(droot, "expression_dataTuck_sub.csv")
dpath_trj_data = osp.join(droot, "pseudotimeTuck.txt")
dpath_branch_data = osp.join(droot, "cell_selectTuck.txt")
dpath_tf_data = osp.join(droot, "mouse_tfs.txt")
spath_result_matrix = osp.join(droot, "TE_result_matrix.txt")

In [ ]:
worker = fte.FastTENET(dpath_exp_data=dpath_exp_data,
                       dpath_trj_data=dpath_trj_data,
                       dpath_branch_data=dpath_branch_data,
                       dpath_tf_data=dpath_tf_data,
                       spath_result_matrix=spath_result_matrix,
                       make_binary=True)

Binary file already exist, load data from binary file


In [ ]:
result_matrix = worker.run(backend="gpu", device_ids=1, procs_per_device=1, batch_size=2**16, binning_method="FSBW-L", kp=0.5)

[DISCRETIZER: FSBW-L, SMOOTHER: None]
[GPU device selected]
[Num. GPUS: 1, Num. Pairs: 616640, Num. GPU_Pairs: 616640, Batch Size: 65536, Process per device: 1]
Total processing elapsed time 19.431766510009766sec.
Save result matrix: ./TE_result_matrix.txt


# Make GRN

### parameters
- **fp_rm**: result matrix, required
- **fp_tf**: tf list file, optional
- **fdr**: specifying fdr, optional, default: 0.01
- **t_degrees**: specifying number of outdegrees, optional, generate final GRNs by incrementally increasing the fdr \
value until the total number of outdegrees is greater than the parameter value.
- **trim_threshold**: trimming threshold, optional, default: 0


In [ ]:
!python ../stats/make_grn.py --fp_rm TE_result_matrix.txt --fp_tf mouse_tfs.txt --fdr 0.01 --trim_threshold 0

Number of genes:  3281
/content/fasttenet/tutorials/../stats/make_grn.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ix = int(ix)
Number of pairs:  616640
fdr: 0.01
Degrees: 3593
save grn in  TE_result_matrix.byGRN.fdr0.01.sif
save trimmed grn in  TE_result_matrix.byGRN.fdr0.01.trimIndirect0.0.sif
save grn outdegrees in  TE_result_matrix.byGRN.fdr0.01.sif.outdegrees.txt
save trimmed grn outdegrees in  TE_result_matrix.byGRN.fdr0.01.trimIndirect0.0.sif.outdegrees.txt


# Run FastTENET with other acceleration frameworks

### PyTorch

In [ ]:
result_matrix = worker.run(backend="torch", device_ids=1, procs_per_device=1, batch_size=2**16, binning_method="FSBW-L", kp=0.5)

[DISCRETIZER: FSBW-L, SMOOTHER: None]
[GPU device selected]
[Num. GPUS: 1, Num. Pairs: 616640, Num. GPU_Pairs: 616640, Batch Size: 65536, Process per device: 1]
Total processing elapsed time 18.685927867889404sec.
Save result matrix: ./TE_result_matrix.txt


### CuPy

In [ ]:
result_matrix = worker.run(backend="cupy", device_ids=1, procs_per_device=1, batch_size=2**16, binning_method="FSBW-L", kp=0.5)

[DISCRETIZER: FSBW-L, SMOOTHER: None]
[GPU device selected]
[Num. GPUS: 1, Num. Pairs: 616640, Num. GPU_Pairs: 616640, Batch Size: 65536, Process per device: 1]
Total processing elapsed time 618.3516731262207sec.
Save result matrix: ./TE_result_matrix.txt


### JAX

In [ ]:
result_matrix = worker.run(backend="jax", device_ids=1, procs_per_device=1, batch_size=2**16, binning_method="FSBW-L", kp=0.5)

[DISCRETIZER: FSBW-L, SMOOTHER: None]
[GPU device selected]
[Num. GPUS: 1, Num. Pairs: 616640, Num. GPU_Pairs: 616640, Batch Size: 65536, Process per device: 1]
Total processing elapsed time 194.12233209609985sec.
Save result matrix: ./TE_result_matrix.txt


### TensorFlow

In [ ]:
result_matrix = worker.run(backend="tf", device_ids=1, procs_per_device=1, batch_size=2**16, binning_method="FSBW-L", kp=0.5)

[DISCRETIZER: FSBW-L, SMOOTHER: None]
[GPU device selected]
[Num. GPUS: 1, Num. Pairs: 616640, Num. GPU_Pairs: 616640, Batch Size: 65536, Process per device: 1]
Total processing elapsed time 178.52658414840698sec.
Save result matrix: ./TE_result_matrix.txt


### NumPy

In [ ]:
result_matrix = worker.run(backend="cpu", device_ids=1, procs_per_device=1, batch_size=2**16, binning_method="FSBW-L", kp=0.5)